# Test2_age_band을 더미변수화하여 머신러닝 실행
최저 정확도 60.6%

-----------

# [T-Academy X KaKr] 성인 인구조사 소득 예측 대회
#### https://www.kaggle.com/c/kakr-4th-competition

- id
- age : 나이
- workclass : 고용 형태
- fnlwgt : 사람 대표성을 나타내는 가중치 (final weight의 약자)
- education : 교육 수준
- education_num : 교육 수준 수치
- marital_status: 결혼 상태
- occupation : 업종
- relationship : 가족 관계
- race : 인종
- sex : 성별
- capital_gain : 양도 소득
- capital_loss : 양도 손실
- hours_per_week : 주당 근무 시간
- native_country : 국적
- income : 수익 (예측해야 하는 값)
- \>50K : 1
- <=50K : 0

## 1. age 속성만을 이용하여 머신러닝 사용해보기_20201208


&nbsp;캐글의 <b>타이타닉 생존자 예측</b>에서 복붙 수준으로 응용해본 게 전부라 머신러닝에 대해서 너무나도 대략적으로만 알고 있기에,<br>
타이타닉 생존자 예측과 같이 이진 분류(?) 형태의 결과를 산출하는 이번 <b>소득 예측 대회</b>를 통해 머신러닝에 대해 조금 더 구체적으로 학습하고자 한다.

------------

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

데이터 정제를 위한 라이브러리와 머신러닝 적용을 위한 라이브러리

In [2]:
train = pd.read_csv("C:/Users/junyoon/Desktop/jun/Kaggle/kakr-4th-competition/train.csv")
test = pd.read_csv("C:/Users/junyoon/Desktop/jun/Kaggle/kakr-4th-competition/test.csv")

train 데이터와 test 데이터 로드

In [3]:
train.head()

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,0,40,Private,168538,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,60,United-States,>50K
1,1,17,Private,101626,9th,5,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,20,United-States,<=50K
2,2,18,Private,353358,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,16,United-States,<=50K
3,3,21,Private,151158,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,25,United-States,<=50K
4,4,24,Private,122234,Some-college,10,Never-married,Adm-clerical,Not-in-family,Black,Female,0,0,20,?,<=50K


In [4]:
test.head()

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
0,0,28,Private,67661,Some-college,10,Never-married,Adm-clerical,Other-relative,White,Female,0,0,40,United-States
1,1,40,Self-emp-inc,37869,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States
2,2,20,Private,109952,Some-college,10,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,25,United-States
3,3,40,Private,114537,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States
4,4,37,Private,51264,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,99,France


데이터 살펴보기

In [5]:
'''
train.loc[train.income == '>50K'] = 1
이렇게 삽입할 경우, income 값이 만족하는 경우 행 내 모든 속성의 값이 1을 갖게 됨.
'''

train.loc[train['income']=='>50K','income'] = 1
train.loc[train['income']=='<=50K','income'] = 0

머신러닝 적용을 위해 label이 되는 income의 텍스트값을 숫자형인 1과 0으로 치환

In [6]:
train_data = train[['age','income']]
train_data.head()

,age,income
0,40,1
1,17,0
2,18,0
3,21,0
4,24,0


In [7]:
test_data = test['age']
test_data.head()

0    28
1    40
2    20
3    40
4    37
Name: age, dtype: int64

이번 학습에서는 age속성만을 사용할 것이므로, train에서는 age와 income을, test에서는 age만을 추출

In [8]:
train_data.age.describe()

count    26049.000000
mean        38.569235
std         13.671489
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: age, dtype: float64

In [9]:
test_data = pd.DataFrame(test_data, columns = ['age'])

In [10]:
test_data.age.describe()

count    6512.000000
mean       38.631296
std        13.516418
min        17.000000
25%        28.000000
50%        37.000000
75%        48.000000
max        90.000000
Name: age, dtype: float64

연속형의 age를 범주형으로 나타내기 위해, 우선 최소값과 최대값을 확인<br>
test_data의 경우, 속성 하나만을 가져오면서 데이터형이 series형으로 바뀌었으므로 다시 데이터프레임으로 변환

In [11]:
train_data['age_band']=0

<ipython-input-11-787d3ccb5db7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['age_band']=0


In [12]:
train_data['age_band'] = pd.cut(train_data['age'], bins = [15,25,35,45,55,65,75,85,95], labels = [1,2,3,4,5,6,7,8])
train_data.head()

<ipython-input-12-a3556df5e30a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['age_band'] = pd.cut(train_data['age'], bins = [15,25,35,45,55,65,75,85,95], labels = [1,2,3,4,5,6,7,8])


,age,income,age_band
0,40,1,3
1,17,0,1
2,18,0,1
3,21,0,1
4,24,0,1


In [13]:
train_data.loc[(train_data['age']>=15)&(train_data['age']<25),'age_band']=1
train_data.loc[(train_data['age']>=25)&(train_data['age']<35),'age_band']=2
train_data.loc[(train_data['age']>=35)&(train_data['age']<45),'age_band']=3
train_data.loc[(train_data['age']>=45)&(train_data['age']<55),'age_band']=4
train_data.loc[(train_data['age']>=55)&(train_data['age']<65),'age_band']=5
train_data.loc[(train_data['age']>=65)&(train_data['age']<75),'age_band']=6
train_data.loc[(train_data['age']>=75)&(train_data['age']<85),'age_band']=7
train_data.loc[(train_data['age']>=85)&(train_data['age']<95),'age_band']=8
train_data.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,age,income,age_band
0,40,1,3
1,17,0,1
2,18,0,1
3,21,0,1
4,24,0,1


In [14]:
test_data['age_band']=0

In [15]:
test_data['age_band'] = pd.cut(test_data['age'], bins = [15,25,35,45,55,65,75,85,95], labels = [1,2,3,4,5,6,7,8])
test_data.head()

,age,age_band
0,28,2
1,40,3
2,20,1
3,40,3
4,37,3


In [16]:
test_data.loc[(test_data['age']>=15)&(test_data['age']<25),'age_band']=1
test_data.loc[(test_data['age']>=25)&(test_data['age']<35),'age_band']=2
test_data.loc[(test_data['age']>=35)&(test_data['age']<45),'age_band']=3
test_data.loc[(test_data['age']>=45)&(test_data['age']<55),'age_band']=4
test_data.loc[(test_data['age']>=55)&(test_data['age']<65),'age_band']=5
test_data.loc[(test_data['age']>=65)&(test_data['age']<75),'age_band']=6
test_data.loc[(test_data['age']>=75)&(test_data['age']<85),'age_band']=7
test_data.loc[(test_data['age']>=85)&(test_data['age']<95),'age_band']=8
test_data.head()

,age,age_band
0,28,2
1,40,3
2,20,1
3,40,3
4,37,3


age_band라는 빈 속성을 만든 뒤, age값 조건에 따라 age_band에 값을 넣는 방식으로 범주화
값을 넣는 방식은 cut함수를 이용한 방법과 loc함수를 이용한 방법, 두 가지를 사용할 수 있음

In [17]:
train_data = train_data[['age_band','income']]
train_data.head()

,age_band,income
0,3,1
1,1,0
2,1,0
3,1,0
4,1,0


In [18]:
test_data = test_data['age_band']
test_data.head()

0    2
1    3
2    1
3    3
4    3
Name: age_band, dtype: category
Categories (8, int64): [1 < 2 < 3 < 4 < 5 < 6 < 7 < 8]

In [19]:
test_data = pd.DataFrame(test_data, columns = ['age_band'])
test_data.head()

,age_band
0,2
1,3
2,1
3,3
4,3


다시 필요한 속성만을 추출하여 새로운 데이터프레임 생성

In [20]:
type(train_data.iloc[3,1])

int

In [21]:
train_label = train_data['income']
train_data = train_data.drop('income', axis = 1)

In [22]:
train_label = train_label.astype('int')

label과 train데이터를 분리하고, 머신러닝에 적용하기 위해 income의 속성값을 int형으로 변환

In [23]:
train_data['age_band'] = train_data['age_band'].astype('str')

In [24]:
train_data = pd.get_dummies(train_data, columns=['age_band'])

In [25]:
test_data = pd.get_dummies(test_data, columns=['age_band'])

In [26]:
def train_and_test(model):
    model.fit(train_data, train_label)
    prediction = model.predict(test_data)
    accuracy = round(model.score(train_data, train_label) * 100, 2)
    print('Accuracy:', accuracy, "%")
    return prediction


머신러닝 적용을 위한 함수 생성

In [27]:
# Logistic Regression
log_pred = train_and_test(LogisticRegression())
# SVM
svm_pred = train_and_test(SVC())
# kNN
knn_pred_4 = train_and_test(KNeighborsClassifier(n_neighbors=4))
# Random Forest
rf_pred = train_and_test(RandomForestClassifier(n_estimators=100))
# Naive Bayes
nb_pred = train_and_test(GaussianNB())

Accuracy: 75.8 %
Accuracy: 75.8 %
Accuracy: 75.8 %
Accuracy: 75.8 %
Accuracy: 60.59 %


함수에 각 머신러닝을 적용하고 정확도 표시

In [32]:
submission = pd.DataFrame({
    'id' : test['id'],
    'prediction': log_pred
})

submission.to_csv('submission.csv', index=False)

정확도가 가장 높은 모델의 예측값을 적용하여 결과 csv 파일 생성

In [29]:
submission1 = pd.DataFrame({
    'id' : test['id'],
    'prediction':log_pred
})
submission2 = pd.DataFrame({
    'id' : test['id'],
    'prediction':svm_pred
})
submission3 = pd.DataFrame({
    'id' : test['id'],
    'prediction':knn_pred_4
})
submission4 = pd.DataFrame({
    'id' : test['id'],
    'prediction':rf_pred
})
submission5 = pd.DataFrame({
    'id' : test['id'],
    'prediction':nb_pred
})

In [30]:
submission5.prediction.unique()

array([0, 1])

정확도가 75.8% 보다 낮은 모델에서 1 확인